In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim

In [ ]:
num_classes = 25
input_size = 28
sequence_size=28

num_layers = 2
hidden_size = 256

num_epochs = 5
learning_rate = 0.01
batch_size=64

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ShuklaRNN(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes):
        super(ShuklaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size * 2, num_classes)
        
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device=device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device=device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:,1,:]

        out = self.fc1(out)
        return out

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=input_size, num_classes=num_classes):
        super(SimpleNN, self).__init__()
        
#         print(type(input_size))
        
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out
        
        

In [ ]:
model = ShuklaRNN().to(device=device)

In [ ]:
# testing
x = torch.randn(64,28,28).to(device=device)
# x = x.reshape(x.size(0), -1)
y = model(x)
print(y.shape)


In [ ]:
import pandas as pd

df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")

df.iloc[:,1:].head().values 

In [ ]:
import numpy as np
np.sort(df['label'].unique()), len(df['label'].unique())


9th label is missing !

In [ ]:
print(len(df), df.iloc[:,1:].head().values.shape)

In [ ]:
class HandSignDataset(Dataset):
    def __init__(self, datapath):
        df = pd.read_csv(datapath)
        self.l = len(df)
        self.x = torch.from_numpy(df.iloc[:,1:].values)
#         print(type(self.x.size))
        self.x = self.x.reshape(self.x.size(0), 1, 28, 28).squeeze(1)
        self.x = self.x.squeeze()
        self.x = self.x.float()
        
        self.y = torch.from_numpy(df.iloc[:,0].values)
        self.y = self.y.long()
    
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.l
        

In [ ]:
train_dataset = HandSignDataset("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test_dataset = HandSignDataset("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

train_dataloader = DataLoader(dataset=train_dataset, batch_size=64,shuffle=True )
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64,shuffle=True )

In [ ]:
x,y = train_dataset[0]
x.shape, y.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(x)
print(y)

In [ ]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate  )

### for saving the data

In [ ]:
def save_checkpoint(state,filename="my_checkpoint.pth.tar"):
    print(f"=> saving to {filename}")
    torch.save(state, filename)

In [ ]:
def load_checkpoint(checkpoint):
    print(f"=> loading from checkpoint {checkpoint}")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    

In [ ]:
load_model = False

In [ ]:
## Loading checkpoint
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

In [ ]:
# train loop

for epoch in range(num_epochs):
    
    if (epoch % 3 == 0):
        checkpoint = {'state_dict': model.state_dict(), 'optimizer':optimizer.state_dict()}
        save_checkpoint(checkpoint)
        
    for x, y in train_dataloader:
        
        x = x.reshape(x.size(0), 1, 28, 28).squeeze(1).to(device=device)
        y = y.to(device=device)
        
        pred = model(x)
        loss = loss_criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    print(f"loss for epoch {epoch} : {loss}")

In [ ]:
def check_accuracy(m, input_dataloader):
    total_sample = 0
    correct_sample = 0
    
    m.eval()
    
    with torch.no_grad():
        for x, y in input_dataloader:
            x = x.reshape(x.size(0), 1, 28, 28).squeeze(1).to(device=device)
            x = x.to(device=device)
            y = y.to(device=device)
            
            preds = m(x)
            _,predictions = preds.max(1)
            
            correct_sample = (y==predictions).sum()
            total_sample += predictions.size(0)
    m.train()
    
    print(f"out of total sample : {total_sample}  correct sample : {correct_sample} accuracy : {float(correct_sample/total_sample)*100:.2f}")
            

In [ ]:
check_accuracy(model, train_dataloader)
check_accuracy(model, test_dataloader)

The reason for low accuracy seems to stem from the fact that label 9 is missing.
Any idea how to improve?